In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from torch.utils.data import random_split

from nam.wrapper import NAMRegressor, MultiTaskNAMRegressor

/home/lemeln/anaconda3/envs/nam37/lib/python3.7/site-packages/torch-1.7.0-py3.7-linux-x86_64.egg/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
housing = fetch_california_housing()

In [3]:
df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
df["target"] = housing.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
scaler = MinMaxScaler((-1, 1))
data = scaler.fit_transform(df)

In [5]:
data

array([[ 0.07933684,  0.56862745, -0.9129754 , ...,  0.13496281,
        -0.57768924,  0.80453276],
       [ 0.07605412, -0.21568627, -0.92355209, ...,  0.13071201,
        -0.57569721,  0.41649313],
       [-0.06794389,  1.        , -0.89448707, ...,  0.12858661,
        -0.57968127,  0.39010148],
       ...,
       [-0.83447125, -0.37254902, -0.93819229, ...,  0.46439957,
        -0.37649402, -0.6812343 ],
       [-0.8114095 , -0.33333333, -0.93643463, ...,  0.46439957,
        -0.39641434, -0.71257438],
       [-0.73949325, -0.41176471, -0.93749509, ...,  0.45164718,
        -0.38047809, -0.69319302]])

In [6]:
data_train, data_test = train_test_split(data, train_size=0.8, test_size=0.2)
X_train, y_train = data_train[:,:-1], data_train[:,-1]
X_test, y_test = data_test[:,:-1], data_test[:,-1]

In [7]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((16512, 8), (16512,), (4128, 8), (4128,))

In [11]:
model = NAMRegressor(
            num_epochs=100,
            num_learners=2,
            metric='mse',
            monitor_loss=True,
            n_jobs=2,
            lr=0.00674,
            output_reg=0.001,
            l2_reg=1e-6,
            dropout=0.0,
            feature_dropout=0.0
        )
model.fit(X_train, y_train)

pred = model.predict(X_test)

In [13]:
float(np.sqrt(sk_metrics.mean_squared_error(y_test, pred)))

0.28242692522239754

In [10]:
data_train, data_test = train_test_split(data, train_size=0.8, test_size=0.2)
X_train, y_train = data_train[:,:-2], data_train[:,-2:]
X_test, y_test = data_test[:,:-2], data_test[:,-2:]

In [11]:
model = MultiTaskNAMRegressor(
            num_epochs=100,
            num_learners=2,
            metric='mse',
            monitor_loss=True,
            n_jobs=2,
            num_subnets=2
        )
model.fit(X_train, y_train)

pred = model.predict(X_test)

In [12]:
float(np.sqrt(sk_metrics.mean_squared_error(y_test[:,-1], pred[:,-1])))

0.3353545074194364